In this project, I'll be working with messy medical data and using regex to extract relevant infromation from the data.

Each line of the dates.txt file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates.

Here is a list of some of the date variants in this dataset:

    04/20/2009; 04/20/09; 4/20/09; 4/3/09
    Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
    20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    Feb 2009; Sep 2009; Oct 2010
    6/2008; 12/2009
    2009; 2010

Once date patterns have been extracted, the next step is to sort them in ascending chronological order accoring to the following rules:

    All dates in xx/xx/xx format are mm/dd/yy
    All dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
    If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
    If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
    Watch out for potential typos as this is a raw, real-life derived dataset.

My goal is to find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

0   1999

1   2010

2   1978

3   2015

4   1985

The final series should look like this:

0    2

1    4

2    0

3    1

4    3

In [1]:
import pandas as pd

doc = []
with open("D:\Python\data\dates.txt") as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [10]:
import re
dates_df = pd.DataFrame(df)

# extracting dates with nonoverlapping patterns
date_formats = [r'(\d{1,2}[-/]\d{1,2}[-/]\d{2,4})',
                r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,.\s-]+\d{1,2}[-,\s]+\d{4})',
                r'(\d{1,2}\s+(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.,\s]+\d{4})',
                r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s+\d{1,2}(?:st|nd|th),\s+\d{4})',
                # r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,.\s]+\d{4})',
                # r'(\d{1,2}/\d{4})',
                #r'(\d{4})'
               ]
date_pattern = r'|'.join(date_formats)
dates_df['dates_str'] = pd.Series(dates_df[0].str.extract(date_pattern).stack().droplevel(1))

# extracting remaining date patterns
dates_df['dates1'] = dates_df[0].str.extract(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,.\s]+\d{4})')
dates_df['dates2'] = dates_df[0].str.extract(r'(\d{1,2}/\d{4})')
dates_df['dates3'] = dates_df[0].str.extract(r'(\d{4})')

# saving all extracted dates in one column
dates_df['dates_str'] = dates_df['dates_str'].fillna(dates_df['dates1']).fillna(dates_df['dates2']).fillna(dates_df['dates3'])

dates_df[271:]

,0,dates_str,dates1,dates2,dates3
271,.Spoke to sister Naomi Ely 708-810-7787 who re...,August 2008,August 2008,NaN,7787
272,sShe tries to follow several dietary rules inc...,Feb 1993,Feb 1993,NaN,1993
273,") - Zoloft 100 mg daily: February, 2010 : self...","February, 2010","February, 2010",NaN,2010
274,"7HH, April 1985 Hx of Outpatient Treatment:\n",April 1985,April 1985,NaN,1985
275,". Ex-BF 25 yo, plans to go to sch in URUGUAY...",September 1984,September 1984,NaN,1984
...,...,...,...,...,...
495,1979 Family Psych History: Family History of S...,1979,NaN,NaN,1979
496,therapist and friend died in ~2006 Parental/Ca...,2006,NaN,NaN,2006
497,2008 partial thyroidectomy\n,2008,NaN,NaN,2008
498,sPt describes a history of sexual abuse as a c...,2005,NaN,NaN,2005


In [11]:
# fixing typos
dates_df['dates_str'][298] = 'January 1993'
dates_df['dates_str'][313] = 'December 1978'

# extracing year from each date string
dates_df['year1'] = dates_df['dates_str'].str.extract(r'.*(\d{2,4})')
dates_df['year'] = dates_df['dates_str'].str.extract(r'(\d{4})').fillna(dates_df['year1'])

# setting all years encoded in only two digits to years from the 1900's 
dates_df['year'] = dates_df['year'].astype(int).apply(lambda x: x if x>100 else x+1900)

# dropping unnecessary columns
dates_df = dates_df[[0,'dates_str', 'year']]
dates_df

,0,dates_str,year
0,03/25/93 Total time of visit (in minutes):\n,03/25/93,1993
1,6/18/85 Primary Care Doctor:\n,6/18/85,1993
2,sshe plans to move as of 7/8/71 In-Home Servic...,7/8/71,1993
3,7 on 9/27/75 Audit C Score Current:\n,9/27/75,1993
4,2/6/96 sleep studyPain Treatment Pain Level (N...,2/6/96,1993
...,...,...,...
495,1979 Family Psych History: Family History of S...,1979,1979
496,therapist and friend died in ~2006 Parental/Ca...,2006,2006
497,2008 partial thyroidectomy\n,2008,2008
498,sPt describes a history of sexual abuse as a c...,2005,2005


In [29]:
# converting date string to datetime
dates_df['dates'] = pd.to_datetime(dates_df['dates_str'])

# setting appropriate years
year = dates_df['year'].astype(str)
month_day = dates_df['dates'].dt.strftime('%m%d')
dates_df['dates'] = pd.to_datetime(year + month_day, format='%Y%m%d')

# Series in chronological order of the original dates_df indices
ordered_date_series = pd.Series(dates_df['dates'].sort_values(kind='stable').index)
ordered_date_series


0      474
1      153
2      129
3      225
4      171
      ... 
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64